# API Project
## Foursquare data analysis

#### Md. Nazmul Haque Khan, Hoda Esfahani

In [222]:
import requests
import pandas as pd
import json

### First step: getting 50 recommendations around the location. (Will do this for 10 cities)

#### First we test it for Helsinki:

In [267]:
response = requests.get('https://api.foursquare.com/v2/venues/explore?ll=60.192059,24.945831&limit=50&client_id={}&client_secret={}&oauth_token={}&v={}'.format(
    'CJIPDECE1LFA05ZH3X4VLTBLWT0YPDC1RM33ES3W1YJYNKGF',
    'A4JQ0RXRSCHCWXOOTU4O1YOI5NF4EX34LLSN23ML0O4VI1UZ',
    'TY5RPWBPNW2CMI5Y04D0XHNVL3TRXGUAYLA3VBF2ILSJOQKL',
    '20200703'
))

In [268]:
res = response.json()

#### Info of each venue, in the recommendation list:

In [269]:
res['response']['groups'][0]['items'][0]['venue']

{'id': '4bcc34bc937ca5937cafaa92',
 'name': 'Lenininpuisto',
 'location': {'address': 'Vesilinnankatu',
  'crossStreet': 'Tivolitie',
  'lat': 60.18991605810939,
  'lng': 24.943149089813232,
  'labeledLatLngs': [{'label': 'display',
    'lat': 60.18991605810939,
    'lng': 24.943149089813232}],
  'distance': 280,
  'cc': 'FI',
  'city': 'Helsinki',
  'state': 'Uusimaa',
  'country': 'Suomi',
  'formattedAddress': ['Vesilinnankatu (Tivolitie)', 'Helsinki']},
 'categories': [{'id': '4bf58dd8d48988d163941735',
   'name': 'Park',
   'pluralName': 'Parks',
   'shortName': 'Park',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
    'suffix': '.png'},
   'primary': True}],
 'photos': {'count': 0, 'groups': []}}

In [ ]:
res

#### We made a basic DataFrame from the information of the recommendation list we have, only keeping the desired fields. This won't be our final DataFrame, but we will use the ids of the venues for our final one:

In [340]:
helsinki_df = pd.json_normalize(res['response']['groups'][0]['items'])
helsinki_df.drop(['referralId', 'reasons.count', 'reasons.items', 'venue.location.address', 'venue.location.cc', 'venue.location.crossStreet', 'venue.photos.count', 'venue.location.labeledLatLngs', 'venue.location.distance', 'venue.location.state', 'venue.location.formattedAddress', 'venue.photos.groups', 'venue.location.postalCode', 'venue.location.neighborhood', 'venue.venuePage.id'], axis=1, inplace=True)
helsinki_df.columns = ['id', 'name', 'latitude', 'longitude', 'city', 'country', 'category']
helsinki_df['category'] = helsinki_df['category'].astype(str).str.extract('(?<=\'name\': \')(.*)(?=\', \'p)', expand=True)
helsinki_df.head()

,id,name,latitude,longitude,city,country,category
0,4bcc34bc937ca5937cafaa92,Lenininpuisto,60.189916,24.943149,Helsinki,Suomi,Park
1,5cf5c8c96fd626002c4fbf65,Konepaja Biergarten,60.192886,24.947617,Helsinki,Suomi,Beer Garden
2,4b1809a0f964a520dccb23e3,Maltainen Riekko,60.192405,24.945079,Helsinki,Suomi,Pub
3,55f32b4d498ed2c829de7b7d,Alppilan Huone,60.190893,24.945556,Helsinki,Suomi,Bar
4,5a240d39da5ede3855f51f73,Going Greek,60.189086,24.948091,Helsinki,Suomi,Greek Restaurant


#### We have chosen these European cities besides Helsinki for our analysis, and we are going to use their coordinates to get 50 recommended places in each:
    Paris: 48.85341,2.3488
    Prague: 50.073658,14.418540
    London: 51.50853,-0.12574
    Barcelona: 41.390205,2.154007
    Rome: 41.902782,12.496366
    Amsterdam: 52.377956,4.897070
    Vienna: 48.210033,16.363449
    Budapest: 47.49801,19.03991
    Zurich: 47.3769,8.5417

In [308]:
ll_list = ['60.192059,24.945831', '48.85341,2.3488', '50.073658,14.418540', '51.50853,-0.12574', '41.390205,2.154007', '41.902782,12.496366', '52.377956,4.897070', '48.210033,16.363449', '47.49801,19.03991', '47.3769,8.5417']
rec_list = []
for ll in ll_list:
    response = requests.get('https://api.foursquare.com/v2/venues/explore?ll={}&limit=50&client_id={}&client_secret={}&oauth_token={}&v={}'.format(
    ll,
    'CJIPDECE1LFA05ZH3X4VLTBLWT0YPDC1RM33ES3W1YJYNKGF',
    'A4JQ0RXRSCHCWXOOTU4O1YOI5NF4EX34LLSN23ML0O4VI1UZ',
    'TY5RPWBPNW2CMI5Y04D0XHNVL3TRXGUAYLA3VBF2ILSJOQKL',
    '20200703'
    ))
    rec_list.append(response.json())

In [309]:
len(rec_list)

10

In [341]:
venues = []
for item in rec_list:
    for city in item['response']['groups']:
        for venue in city['items']:
            venues.append(venue['venue'])
cities_rec = pd.json_normalize(venues)
cities_rec.drop(['location.address', 'location.cc', 'location.crossStreet', 'photos.count', 'location.labeledLatLngs', 'location.state', 'location.formattedAddress', 'photos.groups', 'location.postalCode', 'location.neighborhood', 'venuePage.id'], axis=1, inplace=True)
cities_rec.columns = ['id', 'name', 'category', 'latitude', 'longitude', 'distance_from_spot', 'city', 'country']
cities_rec['category'] = cities_rec['category'].astype(str).str.extract('(?<=\'name\': \')(.*)(?=\', \'p)', expand=True)
cities_rec.tail()

,id,name,category,latitude,longitude,distance_from_spot,city,country
495,539bfde0498e79c67467cb43,Le Prêt-à-manger,Bakery,47.377789,8.538997,226,Zürich Hauptbahnhof,Schweiz
496,4c4c1334959220a190a07810,Polyterrasse,Plaza,47.375932,8.546564,382,Zürich,Schweiz
497,4b54bb21f964a5206ac927e3,Raclette-Stube,Swiss Restaurant,47.374807,8.544690,324,Zürich,Schweiz
498,4b058887f964a520b0cb22e3,Arlette,Hotel,47.378817,8.543579,256,Zürich,Schweiz
499,4c6ff88fd7fab1f738e05cc9,Pestalozzi-Wiese,Plaza,47.375602,8.538818,260,NaN,Schweiz


In [475]:
cities_rec['category']

Hotel                 38
Café                  21
Plaza                 18
Italian Restaurant    18
French Restaurant     13
                      ..
Building               1
Exhibit                1
Juice Bar              1
African Restaurant     1
Jazz Club              1
Name: category, Length: 158, dtype: int64

#### We made a list of venue ids, so we can use them in calling the Details endpoint, for getting more detailed information on each venue:

In [351]:
id_list = list(cities_rec['id'])

In [450]:
len(id_list)

500

In [352]:
for venue in id_list:
    details = requests.get('https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(
        venue,
        'CJIPDECE1LFA05ZH3X4VLTBLWT0YPDC1RM33ES3W1YJYNKGF',
        'A4JQ0RXRSCHCWXOOTU4O1YOI5NF4EX34LLSN23ML0O4VI1UZ',
        'TY5RPWBPNW2CMI5Y04D0XHNVL3TRXGUAYLA3VBF2ILSJOQKL',
        '20200703'
    ))
    details_list.append(details.json())
len(details_list)

550

In [453]:
del details_list[:50] #details_list was not empty when we made the call, and we have limited calls so we just deleted the wrong part

#### Then we saved our data in a file so we won't have to do the calls again:

In [455]:
import pickle

f = open("frsqr_data", "wb")
pickle.dump(details_list, f)
f.close()


In [467]:
venues = []
for item in details_list:
    if 'venue' in item['response']:
        venues.append(item['response']['venue'])


### This is the final DataFrame that needs cleaning:

In [502]:
fs_df = pd.json_normalize(venues)
fs_df.tail()

,id,name,canonicalUrl,categories,verified,url,like,dislike,ok,rating,...,page.user.contact.twitter,page.pageInfo.description,hasMenu,hours.seasonalRange.startMonth,hours.seasonalRange.startDay,hours.seasonalRange.endMonth,hours.seasonalRange.endDay,hours.seasonalRange.closed,hours.seasonalRange.displayDateRange,parent.venuePage.id
473,4f47ed18e4b06a774168ec79,Restaurant Schützengasse,https://foursquare.com/v/restaurant-sch%C3%BCt...,"[{'id': '4bf58dd8d48988d158941735', 'name': 'S...",False,NaN,False,False,False,8.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
474,4af57ab3f964a52054f921e3,Zurich Main Station (Zürich Hauptbahnhof),https://foursquare.com/v/z%C3%BCrich-hauptbahn...,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",True,https://www.sbb.ch,False,False,False,8.3,...,RailService,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
475,4b9b7f1bf964a5201a0b36e3,H&M,https://foursquare.com/v/hm/4b9b7f1bf964a5201a...,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",True,http://hm.com,False,False,False,7.9,...,hm,Fashion and quality at the best price in a sus...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
476,4b8e149ff964a520c81733e3,Café Gourmet,https://foursquare.com/v/caf%C3%A9-gourmet/4b8...,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",True,http://www.hotelschweizerhof.com,False,False,False,7.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
477,5412a354498ebdae0ed564d5,Victorinox Flagship Store Zürich,https://foursquare.com/v/victorinox-flagship-s...,"[{'id': '52f2ab2ebcbc57f1066b8b2e', 'name': 'W...",True,http://www.swissarmy.com,False,False,False,8.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [469]:
f = open("dataframe_raw", "wb")
pickle.dump(fs_df, f)
f.close()

#### The information(columns) we are interested in and will keep them are:
    name
    has_contact
    lat
    lon
    city
    country
    category
    tip_count
    has_website
    likes
    rating
    rating_color
    rating_count
    has_menu
    photo_count
    listed_count

In [459]:
for column in fs_df.columns:
    print(column)

id
name
canonicalUrl
categories
verified
url
like
dislike
ok
rating
ratingColor
ratingSignals
allowMenuUrlEdit
createdAt
shortUrl
timeZone
seasonalHours
contact.twitter
location.address
location.crossStreet
location.lat
location.lng
location.labeledLatLngs
location.cc
location.city
location.state
location.country
location.formattedAddress
stats.tipCount
likes.count
likes.groups
likes.summary
beenHere.count
beenHere.unconfirmedCount
beenHere.marked
beenHere.lastCheckinExpiredAt
specials.count
specials.items
photos.count
photos.groups
reasons.count
reasons.items
hereNow.count
hereNow.summary
hereNow.groups
tips.count
tips.groups
listed.count
listed.groups
popular.isOpen
popular.isLocalHoliday
popular.timeframes
pageUpdates.count
pageUpdates.items
inbox.count
inbox.items
attributes.groups
bestPhoto.id
bestPhoto.createdAt
bestPhoto.source.name
bestPhoto.source.url
bestPhoto.prefix
bestPhoto.suffix
bestPhoto.width
bestPhoto.height
bestPhoto.visibility
colors.highlightColor.photoId
colors.hi

In [504]:
test = fs_df.loc[:, [
    'name', 'categories', 'location.city', 'location.country', 'location.lat',
    'location.lng', 'likes.count', 'rating', 'ratingSignals', 'ratingColor', 'contact.phone', 
    'contact.formattedPhone', 'contact.twitter', 'contact.facebook', 'contact.facebookUsername', 
    'contact.instagram', 'contact.facebookName', 'stats.tipCount', 'photos.count', 'listed.count', 'url'
]]
contact = test.loc[:, [
    'contact.phone', 'contact.formattedPhone', 'contact.twitter', 'contact.facebook', 'contact.facebookUsername', 
    'contact.instagram', 'contact.facebookName']]
test['has_contact'] = ~(contact.isna().all(axis = 1))
test.drop([
    'contact.phone', 'contact.formattedPhone', 'contact.twitter', 'contact.facebook', 'contact.facebookUsername', 
    'contact.instagram', 'contact.facebookName'], axis=1, inplace=True)
test['url'] = test['url'].isna()
test['categories'] = test['categories'].astype(str).str.extract(r'(?<=\'name\': \')(.*?)(?=\', \'p)', expand=True)

In [505]:
test.columns = [
    'name', 'category', 'city', 'country', 'lat', 'lon', 'likes', 'rating', 'rating_count', 
    'rating_color', 'tip_count', 'photos_count', 'listed_count', 'has_website', 'has_contact'
]
fs_df = test
fs_df.tail()

,name,category,city,country,lat,lon,likes,rating,rating_count,rating_color,tip_count,photos_count,listed_count,has_website,has_contact
473,Restaurant Schützengasse,Swiss Restaurant,Zürich,Schweiz,47.377154,8.537430,36,8.5,51,73CF42,12,27,15,True,True
474,Zurich Main Station (Zürich Hauptbahnhof),Train Station,Zürich,Schweiz,47.378705,8.538724,1267,8.3,1643,73CF42,235,4746,222,False,True
475,H&M,Clothing Store,Zürich,Schweiz,47.375877,8.539477,28,7.9,36,C5DE35,1,26,9,False,True
476,Café Gourmet,Café,Zürich,Schweiz,47.377094,8.539425,8,7.9,11,C5DE35,6,14,2,False,True
477,Victorinox Flagship Store Zürich,Watch Shop,Zürich,Schweiz,47.374041,8.538962,39,8.7,46,73CF42,6,31,13,False,True


### This is the final DataFrame that we will use for analysis. We save it in a file:

In [506]:
f = open("dataframe_final", "wb")
pickle.dump(fs_df, f)
f.close()


In [508]:
fs_df.describe(include='all')

,name,category,city,country,lat,lon,likes,rating,rating_count,rating_color,tip_count,photos_count,listed_count,has_website,has_contact
count,478,478,459,478,478.000000,478.000000,478.000000,478.000000,478.000000,478,478.000000,478.000000,478.000000,478,478
unique,473,156,14,10,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,2,2
top,Amorino,Hotel,Paris,France,NaN,NaN,NaN,NaN,NaN,73CF42,NaN,NaN,NaN,False,True
freq,4,36,50,50,NaN,NaN,NaN,NaN,NaN,338,NaN,NaN,NaN,315,349
mean,NaN,NaN,NaN,NaN,48.999710,10.606440,192.543933,8.399372,244.133891,NaN,43.018828,316.560669,226.196653,NaN,NaN
std,NaN,NaN,NaN,NaN,5.185954,8.044202,583.746313,0.464843,686.372431,NaN,89.704486,1292.392383,499.563662,NaN,NaN
min,NaN,NaN,NaN,NaN,41.387399,-0.129001,1.000000,7.100000,2.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,47.377109,2.346964,22.250000,8.100000,29.250000,NaN,6.000000,24.000000,15.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,48.850984,12.494443,56.500000,8.400000,76.000000,NaN,15.000000,57.500000,53.500000,NaN,NaN
75%,NaN,NaN,NaN,NaN,51.509757,16.366150,145.250000,8.700000,192.500000,NaN,42.000000,174.000000,181.000000,NaN,NaN
